# Notebook trainer cheatsheet: API and CLI

- Automation page
- Recovering the API KEY (Automation page, User page, RestClient)

## Important notice

This notebook various usage of the MISP restAPI.

It should be noted that PyMISP is not required to use the MISP restAPI. We are using PyMISP only to parse the response and inspect the data. So any HTTP client such as curl could do the job a described below.

This command:
```
misp_url = URL + '/events/add'
relative_path = ''

body = {
    "info": "Event"
}

misp = ExpandedPyMISP(misp_url, AUTHKEY, False)
res = misp.direct_call(relative_path, body)
print_result(res)
```

Will yield the same result as this command:
```
!curl \
 -d '{"info": "Event"}' \
 -H "Authorization: ptU1OggdiLLWlwHPO9B3lzpwEND3hL7gH0uEsyYL" \
 -H "Accept: application/json" \
 -H "Content-type: application/json" \
 -X POST 127.0.0.1:8080/events/restSearch
 ```

In [ ]:
from pymisp import ExpandedPyMISP
from pprint import pprint
AUTHKEY = "AY6Qur7V1kyQ1BTefWiiTx7B6KM7ABln1UVpfDKB"
URL = "https://localhost:8443"
misp = ExpandedPyMISP(URL, AUTHKEY, False)

def print_result(result):
    flag_printed = False
    if isinstance(result, list):
        print("Count: %s" % len(result))
        flag_printed = True
        for i in res:
            if 'Event' in i and 'Attribute' in i['Event']:
                print("  - Attribute count: %s" % len(i['Event']['Attribute']))
    elif isinstance(result, dict):
        if 'Attribute' in result:
            print("Count: %s" % len(result['Attribute']))
            flag_printed = True
        elif 'Event' in result and 'Attribute' in result['Event']['Attribute']:
            print("Attribute count: %s" % len(result['Event']['Attribute']))
            flag_printed = True
    if flag_printed:
        print('----------')
    pprint(result)

# Events

## Creation and Edition

In [ ]:
# Creation
endpoint = '/events/add'
relative_path = ''

body = {
    "info": "Event created via the API as an example",
    "threat_level_id": 1,
    "distribution": 0
}

res = misp.direct_call(endpoint + relative_path, body)
print_result(res)

In [ ]:
# Edition 1
endpoint = '/events/edit/'
relative_path = '21'

body = {
    "distribution": 3
#     "sharing_group_id": 1
}

res = misp.direct_call(endpoint + relative_path, body)
print_result(res)

In [ ]:
# Edition 2 - Adding Attribute
endpoint = '/events/edit/'
relative_path = '18'

body = {
    "distribution": 0,
    "Attribute": [
        {
            "value": "9.9.9.9",
            "type": "ip-src"
        }
    ]
}

res = misp.direct_call(endpoint + relative_path, body)
print_result(res)

In [ ]:
# Edition 2 - tagging - The bad way (Fetch the whole event and re-process everything)
endpoint = '/events/edit/'
relative_path = '29'

body = {
    "distribution": 0,
    "EventTag": {
        "Tag": [
            {"name":"tlp:red"}
        ]
    }
}

res = misp.direct_call(endpoint + relative_path, body)
print_result(res)

In [ ]:
# Edition 2 - tagging - The better way
endpoint = '/tags/attachTagToObject'
relative_path = ''

body = {
    "uuid": "5d6f857e-698c-4ea0-834a-6db1cfc4a0a0", # can be anything: event or attribute
    "tag": "tlp:green"
}

res = misp.direct_call(endpoint + relative_path, body)
print_result(res)

In [ ]:
# Searching the Event index (Move it to the search topic)
endpoint = '/events/index'
relative_path = ''

body = {
#     "eventinfo": "api",
    "publish_timestamp": "2019-05-21",
#     "org": "ORGNAME"
}

res = misp.direct_call(endpoint + relative_path, body)
print_result(res)

In [ ]:
# Searching the Event index
misp_url = '/events/index'
relative_path = ''

body = {
    "hasproposal": 1,
    "tag": ["tlp:amber"]
}

res = misp.direct_call(endpoint + relative_path, body)

print('Event number: %s' % len(res))
print_result(res)

# Attributes

## Creation and edition

In [ ]:
event_id = XXXXX

In [ ]:
# Adding
endpoint = '/attributes/add/'
relative_path = str(event_id)

body = {
    "value": "8.8.8.9",
    "type": "ip-dst"
}

res = misp.direct_call(endpoint + relative_path, body)
print_result(res)

In [ ]:
# Adding invalid attribute type
endpoint = '/attributes/add/'
relative_path = str(event_id)

body = {
    "value": "8.8.8.9",
    "type": "md5"
}

res = misp.direct_call(endpoint + relative_path, body)
print_result(res)

In [ ]:
# Editing
endpoint = '/attributes/edit/'
relative_path = '36586'

body = {
    "value": "127.0.0.1",
    "to_ids": 0,
    "comment": "Comment added via the API",
}

res = misp.direct_call(endpoint + relative_path, body)
print_result(res)

In [ ]:
# Editing with data taken from JSON views. 
# <!> (timestamp) contrast the difference with *PyMISP*
endpoint = '/attributes/edit/'
relative_path = 'XXXXXXXX'

body = {
                "id": "XXXXXXXX",
                "type": "ip-dst",
                "category": "Network activity",
                "to_ids": False,
                "uuid": "5cf65823-d22c-45ae-af4f-47d80a00020f",
                "event_id": "33",
                "distribution": "5",
                "comment": "Comment added via the API",
                "sharing_group_id": "0",
                "deleted": False,
                "disable_correlation": False,
                "object_id": "0",
                "object_relation": '',
                "value": "1.2.3.5",
                "Galaxy": [],
                "ShadowAttribute": [],
                "Tag": [
                    {
                        "id": "4",
                        "name": "tlp:green",
                        "colour": "#14ff00",
                        "exportable": True,
                        "user_id": "0",
                        "hide_tag": False,
                        "numerical_value": ''
                    }
                ]
            }

res = misp.direct_call(endpoint + relative_path, body)
print_result(res)

# Objects

In [ ]:
# Example of an un-documented endpoint
endpoint = '/objects/add/'
relative_path = str(event_id)

body = {
    "name": "microblog",
    "meta-category": "misc",
    "description": "Microblog post like a Twitter tweet or a post on a Facebook wall.",
    "template_uuid": "8ec8c911-ddbe-4f5b-895b-fbff70c42a60",
    "template_version": "5",
    "event_id": event_id,
    "timestamp": "1558702173",
    "distribution": "5",
    "sharing_group_id": "0",
    "comment": "",
    "deleted": False,
    "ObjectReference": [],
    "Attribute": [
        {
            "type": "text",
            "category": "Other",
            "to_ids": False,
            "event_id": event_id,
            "distribution": "5",
            "timestamp": "1558702173",
            "comment": "",
            "sharing_group_id": "0",
            "deleted": False,
            "disable_correlation": False,
            "object_relation": "post",
            "value": "post",
            "Galaxy": [],
            "ShadowAttribute": []
        }
    ]
}

res = misp.direct_call(endpoint + relative_path, body)
print_result(res)

In [ ]:
# Go to event Edit 2
# Go to add tag the bad way

## RestSearch
**Aka: Most powerful search tool in MISP**

### RestSearch - Attributes

In [ ]:
endpoint = '/attributes/restSearch/'
relative_path = ''

body = {
    "returnFormat": "json",
    "eventid": event_id
}

res = misp.direct_call(endpoint + relative_path, body)
print_result(res)

In [ ]:
# Searches on Attribute's data
misp_url = '/attributes/restSearch/'
relative_path = ''

body = {
    "returnFormat": "json",
    "eventid": event_id,
    "type": "ip-dst",
    "value": "1.2.3.%"
}

res = misp.direct_call(endpoint + relative_path, body)
print_result(res)

In [ ]:
# Searches on Attribute's data
endpoint = '/attributes/restSearch/'
relative_path = ''

body = {
    "returnFormat": "json",
    "eventid": event_id,
    "deleted": [0, 1]    # Consider both deleted AND not deleted
}

# [] == {"OR": []}

res = misp.direct_call(endpoint + relative_path, body)
print_result(res)

In [ ]:
# Searches on Attribute's data
endpoint = '/attributes/restSearch/'
relative_path = ''

body = {
    "returnFormat": "json",
    "eventid": event_id,
#     "tags": "tlp:white",
#     "tags": ["tlp:white", "tlp:green"]
#     "tags": ["!tlp:green"]
#     "tags": "tlp:%",
#     "includeEventTags": 1
#         BRAND NEW (only tag)! Prefered way (Most accurate): Distinction between OR and AND!
    "tags": {"AND": ["tlp:green", "Malware"], "NOT": ["%ransomware%"]}
}

res = misp.direct_call(endpoint + relative_path, body)
print_result(res)

In [ ]:
# Paginating
endpoint = '/attributes/restSearch/'
relative_path = ''

body = {
    "returnFormat": "json",
    "eventid": event_id,
    "page": 2,
    "limit": 1
}

res = misp.direct_call(endpoint + relative_path, body)
print_result(res)

In [ ]:
# Searches based on time: Absolute
endpoint = '/attributes/restSearch/'
relative_path = ''
event_id = 13

body = {
    "returnFormat": "json",
    "eventid": event_id,
    "from": "2019/05/21" # or "2019-05-21"
    # from and to NOT REALLY USEFULL.. 
}

res = misp.direct_call(endpoint + relative_path, body)
print_result(res)

In [ ]:
# Searches based on time: Relative
endpoint = '/attributes/restSearch/'
relative_path = ''

# /!\ Last: works on the publish_timestamp -> may be confusing
# Units: days, hours, minutes and secondes
body = {
    "returnFormat": "json",
    "eventid": event_id,
#     "to_ids": 1,
    "last": "2019-08-28"
}

res = misp.direct_call(endpoint + relative_path, body)
print_result(res)

## Precision regarding the different timestamps
- ``publish_timestamp`` = Time at which the event was published
    - Usage: get data that arrived in my system since x
    - E.g.: New data from a feed
- ``timestamp`` = Time of the last modification on the data
    - data was modified in the last x hours
    - E.g.: Last updated data from a feed
- ``event_timestamp``: Used in the Attribute scope
    - Event modified in the last x hours

In [ ]:
# Searches with attachments
endpoint = '/attributes/restSearch/'
relative_path = ''

body = {
    "returnFormat": "json",
    "eventid": event_id,
    "type": "attachment",
#     "withAttachments": 1
}

res = misp.direct_call(endpoint + relative_path, body)
print_result(res)

In [ ]:
# Searches - Others
endpoint = '/attributes/restSearch/'
relative_path = ''

body = {
    "returnFormat": "json",
    "eventid": 31,
    "type": "ip-src",
#     "enforceWarninglist": 1
}

res = misp.direct_call(endpoint + relative_path, body)
print_result(res)

### RestSearch - Events

In [ ]:
# Searching using the RestSearch
endpoint = '/events/restSearch'
relative_path = ''

body = {
    "returnFormat": "json",
    "eventid": 31,
}

res = misp.direct_call(endpoint + relative_path, body)
print_result(res)

In [ ]:
# Searching using the RestSearch - Other return format
!curl \
 -d '{"returnFormat":"rpz","eventid":31}' \
 -H "Authorization: ptU1OggdiLLWlwHPO9B3lzpwEND3hL7gH0uEsyYL" \
 -H "Accept: application/json" \
 -H "Content-type: application/json" \
 -X POST 127.0.0.1:8080/events/restSearch 2> /dev/null

In [ ]:
# Searching using the RestSearch - Other return format
!curl \
 -d '{"returnFormat":"csv","eventid":31}' \
 -H "Authorization: ptU1OggdiLLWlwHPO9B3lzpwEND3hL7gH0uEsyYL" \
 -H "Accept: application/json" \
 -H "Content-type: application/json" \
 -X POST 127.0.0.1:8080/events/restSearch 2> /dev/null

In [ ]:
# Searching using the RestSearch - Filtering
endpoint = '/events/restSearch'
relative_path = ''

body = {
    "returnFormat": "json",
    "value": "parsed-ail.json"
}

res = misp.direct_call(endpoint + relative_path, body)
print_result(res)

In [ ]:
# Searching using the RestSearch
endpoint = '/events/restSearch'
relative_path = ''

body = {
    "returnFormat": "json",
    "org": "CIRCL",
    "id": 33,
    "metadata": 1
}

res = misp.direct_call(endpoint + relative_path, body)
print_result(res)

In [ ]:
# Searching using the RestSearch
endpoint = '/events/restSearch'
relative_path = ''

body = {
    "returnFormat": "json",
    "eventinfo": "%via the API%",
    "published": 1
}

res = misp.direct_call(endpoint + relative_path, body)
print_result(res)

# Sightings

In [ ]:
# Creating sightings
endpoint = '/sightings/add'
relative_path = ''

body = {
#     "id": "36578"
    "value": "parsed-ail.json"
}

res = misp.direct_call(endpoint + relative_path, body)
print_result(res)

In [ ]:
# Searching for sighted elements
endpoint = '/sightings/restSearch/event'
relative_path = ''

body = {
    "returnFormat": "json",
    "id": 33,
    "includeAttribute": 1,
    "includeEvent": 1
}

res = misp.direct_call(endpoint + relative_path, body)
print_result(res)

# Warning lists

In [ ]:
# Checking values against the warining list
endpoint = '/warninglists/checkValue'
relative_path = ''

body = ["8.8.8.8", "yolo", "test"]

res = misp.direct_call(endpoint + relative_path, body)
print_result(res)

# Instance management

In [ ]:
# Creating Organisation
endpoint = '/admin/organisations/add'
relative_path = ''

body = {
    "name": "TEMP_ORG2"
}

res = misp.direct_call(endpoint + relative_path, body)
print_result(res)

In [ ]:
# Creating Users
endpoint = '/admin/users/add'
relative_path = ''

body = {
    "email": "from_api2@admin.test",
    "org_id": 1009,
    "role_id": 3,
    "termsaccepted": 1,
    "change_pw": 0, # User prompted to change the psswd once logged in
    "password": "~~UlTrA_SeCuRe_PaSsWoRd~~"
}

res = misp.direct_call(endpoint + relative_path, body)
print_result(res)

In [ ]:
# Creating Sharing Groups
endpoint = '/sharing_groups/add'
relative_path = ''

body = {
    "name": "TEMP_SG2",
    "releasability": "To nobody",
    "SharingGroupOrg": [
        {
            "name": "ORGNAME",
            "extend": 1
        },
        {
            "name": "CIRCL",
            "extend": 1
        }
    ]
}

res = misp.direct_call(endpoint + relative_path, body)
print_result(res)

In [ ]:
# Server
endpoint = '/servers/add'
relative_path = ''

body = {
    "url": "http://127.0.0.1:80/",
    "name": "Myself",
    "remote_org_id": "2",
    "authkey": "UHwmZCH4QdSKqPVunxTzfSes8n7ibBhUlsd0dmx9"
    
}

res = misp.direct_call(endpoint + relative_path, body)
print_result(res)

In [ ]:
# Server settings
endpoint = '/servers/serverSettings'
relative_path = ''

body = {}

res = misp.direct_call(endpoint + relative_path, body)
print_result(res)

In [ ]:
# Statistics
endpoint = '/users/statistics'
relative_path = ''

body = {}

res = misp.direct_call(endpoint + relative_path, body)
print_result(res)

Not Available:
- misp-module